#### Import necessary library

In [38]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras import layers
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [21]:
train_dir = 'C:\\Users\\xinga\\OneDrive\\文档\\GitHub\\AI-Project\\dataset\\aclImdb\\train'
val_dir = 'C:\\Users\\xinga\\OneDrive\\文档\\GitHub\\AI-Project\\dataset\\aclImdb\\test'

batch_size = 32

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    class_names=['neg', 'pos'])

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    val_dir,
    batch_size=batch_size,
    class_names=['neg', 'pos'])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [32]:
def dataset_to_dataframe(ds):
    texts = []
    labels = []
    for text, label in ds.unbatch().as_numpy_iterator():
        texts.append(text.decode('utf-8'))
        labels.append(label)
    return pd.DataFrame({'text': texts, 'label': labels})

train_df = dataset_to_dataframe(raw_train_ds)
val_df = dataset_to_dataframe(raw_val_ds)

print(train_df.head())
print(val_df.head())

                                                text  label
0  Who knew they could be so funny?? Christopher ...      1
1  I couldn't have been more thrilled; Just eight...      1
2  The best of the seven Sam Fuller movies that I...      1
3  This is one of the most spiritual movies I hav...      1
4  This is a really well made movie. Sumitra Bhav...      1
                                                text  label
0  This film has the worst editing I've ever seen...      0
1  Roger Spottiswoode isn't the worst director, a...      0
2  I didn't know what to expect from this hugely ...      1
3  "The Great Dictator" is, arguably, one of Char...      1
4  There's not a dull moment in Francois Ozon's "...      1


In [35]:
x_train = train_df['text'].str.lower().values
y_train = train_df['label'].values

x_test = val_df['text'].str.lower().values
y_test = val_df['label'].values

In [40]:
# Define vectorization parameters
max_features = 20000
sequence_length = 200
# Create TextVectorization layer
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [41]:
# Fit the TextVectorization layer on the training texts
vectorize_layer.adapt(x_train)

In [42]:
# Apply TextVectorization to transform the text data
x_train_vectorized = vectorize_layer(np.array(x_train))
x_test_vectorized = vectorize_layer(np.array(x_test))

In [43]:
print(x_train_vectorized)
print(x_test_vectorized)

tf.Tensor(
[[  37  672   35 ...    0    0    0]
 [  10  404   26 ...    0    0    0]
 [   2  115    5 ...  556   19  142]
 ...
 [3838 7308  285 ...    0    0    0]
 [  10  199   11 ...    0    0    0]
 [   1   10   14 ...    0    0    0]], shape=(25000, 200), dtype=int64)
tf.Tensor(
[[  11   20   44 ...    0    0    0]
 [2391    1  207 ...    0    0    0]
 [  10  153  118 ...    0    0    0]
 ...
 [ 364   12  465 ...    9 2938  107]
 [  41  156 3418 ...    0    0    0]
 [  15   11   18 ...    0    0    0]], shape=(25000, 200), dtype=int64)


In [46]:
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=128))
model.add(Dropout(0.5))  # Added Dropout layer after Embedding
model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.5))  # Added Dropout layer after SimpleRNN
model.add(Dense(1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
# Train the model
history = model.fit(
    x_train_vectorized,
    y_train,
    validation_data=(x_test_vectorized, y_test),
    epochs=5,
    batch_size=32)

Epoch 1/5
782/782 [==============================] - 76s 96ms/step - loss: 0.7194 - accuracy: 0.5020 - val_loss: 0.6939 - val_accuracy: 0.4950
Epoch 2/5
 44/782 [>.............................] - ETA: 1:05 - loss: 0.7006 - accuracy: 0.5185

KeyboardInterrupt: 

In [22]:
raw_train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [23]:
# Define the text vectorization layer
vectorize_layer = layers.TextVectorization(
    max_tokens=20000,
    output_mode='int',
    output_sequence_length=600)

In [24]:
# Fit the text vectorization layer to the training data
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [25]:
# Define a function to apply the text vectorization layer to the data
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [36]:
# Apply the text vectorization layer to the training and validation data
train_ds = x_train.map(vectorize_text)
val_ds = x_test.map(vectorize_text)

AttributeError: 'numpy.ndarray' object has no attribute 'map'

In [ ]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [27]:
train_ds

<MapDataset element_spec=(TensorSpec(shape=(None, 600), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [29]:
model = Sequential()
model.add(Embedding(input_dim=20000, output_dim=16))
model.add(Dropout(0.5))  # Added Dropout layer after Embedding
model.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.5))  # Added Dropout layer after SimpleRNN
model.add(Dense(1, activation='sigmoid'))
# model = tf.keras.Sequential([
#     layers.Embedding(input_dim=20000, output_dim=16),
#     SimpleRNN(128),
#     layers.Dense(1, activation='sigmoid')
# ])
# model = tf.keras.Sequential([
#     layers.Embedding(input_dim=10000, output_dim=64),
#     layers.GRU(64),
#     layers.Dense(1, activation='sigmoid')
# ])


In [30]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
# Fit the model to the training data
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=8)

Epoch 1/8
782/782 [==============================] - 118s 150ms/step - loss: 0.7239 - accuracy: 0.5016 - val_loss: 0.6943 - val_accuracy: 0.4978
Epoch 2/8
782/782 [==============================] - 117s 150ms/step - loss: 0.6987 - accuracy: 0.5001 - val_loss: 0.6931 - val_accuracy: 0.4979
Epoch 3/8
 21/782 [..............................] - ETA: 1:40 - loss: 0.6980 - accuracy: 0.4688

KeyboardInterrupt: 

In [ ]:
# Plot accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()